In [14]:
# 動態語言和靜態語言最大的不同，就是函數和類的定義，不是編譯時定義的，而是運行時動態創建的。
class Hello(object):
    def hello(self, name='world'):
        return f'Hello, {name}'
        
h = Hello()
print(h.hello())

print(type(Hello))
# Hello是一個 class, 所以他的type 是 type

print(type(h))
# h 是一個 instance, 所以他的type 是 Hello

# class的定義是運行時動態創建的，而創建class的方法就是使用type()函數

Hello, world
<class 'type'>
<class '__main__.Hello'>


In [15]:
# type()函數既可以返回一個對象的類型，又可以創建出新的類型

def fn(self, name='world'):
    return f'Hello, {name}'

Hello2 = type('Hello', (object,), dict(hello=fn))

h2 = Hello2()
print(h2.hello())

print(type(Hello2))

print(type(h2))

Hello, world
<class 'type'>
<class '__main__.Hello'>


In [17]:
# metaclass

# metaclass 可以說是 class 的模板，可以控制 class 的創建行為
# 通常定義 class 然後創建 instance
# 而定義 metaclass 然後創建 class 最後在創建 instance

class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)  # 利用 type 生出 class
    

# 使用 metaclass 就是在繼承的地方加上 metaclass=XXX
class MyList(list, metaclass=ListMetaclass):
    pass

L = MyList()
print(L)
L.add(1)
print(L)

[]
[1]


In [20]:
# metaclass 適合用在有可能操作修改 class 定義的情況，像是ORM
# ORM全稱“Object Relational Mapping”，即對象-關係映射，就是把關係數據庫的一行映射為一個對象，也就是一個類對應一個表，不用直接操作SQL語句。 
# 要編寫一個ORM框架，所有的類都只能動態定義，因為只有使用者才能根據表的結構定義出對應的類來。

# 先假設給 使用者操作定義的接口長這樣
class User(Model):
    # 定義class的屬性映射到列：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

# # 然後創建一個 instance：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# # 寫入db：
u.save()

# 其中，父類Model和屬性類型StringField、IntegerField是由ORM框架提供的，剩下的魔術方法比如save()全部由metaclass自動完成。
# 雖然metaclass的編寫會比較複雜，但ORM的使用者用起來卻異常簡單。

NameError: name 'Model' is not defined

In [30]:
# 首先來定義Field類，它負責保存數據庫表的字段名和字段類型
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)
    
class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')
        
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)
    
class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))
        
class User(Model):
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')
    
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


当用户定义一个class User(Model)时，Python解释器首先在当前类User的定义中查找metaclass，如果没有找到，就继续在父类Model中查找metaclass，找到了，就使用Model中定义的metaclass的ModelMetaclass来创建User类，也就是说，metaclass可以隐式地继承到子类，但子类自己却感觉不到。

在ModelMetaclass中，一共做了几件事情：

排除掉对Model类的修改；

在当前类（比如User）中查找定义的类的所有属性，如果找到一个Field属性，就把它保存到一个__mappings__的dict中，同时从类属性中删除该Field属性，否则，容易造成运行时错误（实例的属性会遮盖类的同名属性）；

把表名保存到__table__中，这里简化为表名默认为类名。

在Model类中，就可以定义各种操作数据库的方法，比如save()，delete()，find()，update等等。

我们实现了save()方法，把一个实例保存到数据库中。因为有表名，属性到字段的映射和属性值的集合，就可以构造出INSERT语句。

In [58]:
class TestMetaClass(type):
    def __new__(cls, name, bases, attrs):
        print(cls)
        print(name)
        print(bases)
        print(attrs)
        return type.__new__(cls, name, bases, attrs)
    
class A(object):
    a = 'abc'
    print(a)
    def __init__(self):
        print(self.a, 'init')
        print('AAA')
    
class B(object, metaclass=TestMetaClass):
    a = 123
    b = 'bcd'
    print(a)
    def __init__(self):
        print(self.a, 'init')
        print('BBB')
        
class Test(A, B):
    b = 'bbb'
    print('TTT')
    
    def __init__(self):
        print(self.a, 'init')
        print('TEST')
        
    def get(self):
        return self.a
    
a = Test()
# print(a.a)
# print(a.b)
# print(a.get())

abc
123
<class '__main__.TestMetaClass'>
B
(<class 'object'>,)
{'__module__': '__main__', '__qualname__': 'B', 'a': 123, 'b': 'bcd', '__init__': <function B.__init__ at 0x10397b6a8>}
TTT
<class '__main__.TestMetaClass'>
Test
(<class '__main__.A'>, <class '__main__.B'>)
{'__module__': '__main__', '__qualname__': 'Test', 'b': 'bbb', '__init__': <function Test.__init__ at 0x101a44e18>, 'get': <function Test.get at 0x1038c07b8>}
abc init
TEST
